In [1]:
#installing pysm
!pip install git+https://github.com/b-thorne/PySM_public.git


In [2]:
import pysm
from pysm.nominal import models
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
import pandas as pd
from astropy import units as u

In [ ]:
nside=1024
r=2
frequency=[85,95,145,155,220,270]
d1_config = models("d1", nside)
s1_config = models("s1", nside)
f1_config = models("f1", nside)
cmb_config = models("c1", nside)
a1_config = models("a1", nside)
sky_config = {'dust' : d1_config, 'synchrotron' : s1_config,'ame':a1_config,'freefree':f1_config,'cmb':cmb_config}
sky = pysm.Sky(sky_config)
total_signal = sky.signal()(frequency)
cmb_signal = sky.cmb(frequency)
synchrotron_signal = sky.synchrotron(frequency)
hp.mollview(cmb_signal[1, 0, :], title = "Dust T @ 100 GHz")
hp.mollview(total_signal[0, 1, :], title = "Total Q @ 10 GHz")
plt.show()

In [6]:
instrument_config = {
    'nside' : 1024,
    'frequencies' : np.array([85,95,145,155,220,270]), #Expected in GHz
    'use_smoothing' : True,
    'beams' : np.array([25.5,22.7,25.5,22.7,13,13]), #Expected in arcmin
    'add_noise' : True,
    'sens_I' : np.array([1.31,1.15,1.78,1.91,4.66,7.99]), #Expected in units uK_RJ
    'sens_P' : np.ones(6),
    'noise_seed' : 1234,
    'use_bandpass' : False,
    'output_units' : 'uK_RJ',
    'output_directory' : './',
    'output_prefix' : 'test',
}
instrument = pysm.Instrument(instrument_config)
instrument.observe(sky)

In [ ]:
def bin2nbit(binary,n):
  while(len(binary)<n):
    binary='0'+binary
  return binary

mp=np.zeros((12,1024,1024))
m=total_signal[3, 0, :]  # T at 155 GHz

for k in range(12):
  pixar='0'*20
  kb=bin2nbit(bin(k)[2:],4)
  pixar=np.array(list(kb+pixar))

  for i in range(1024):
    ib=np.array(list(bin2nbit(bin(i)[2:],10)))

    for index, value in np.ndenumerate(ib):
      print(index)
      pixar[4+2*index[0]]=value

    for j in range(1024):
      jb=np.array(list(bin2nbit(bin(j)[2:],10)))

      for index, value in np.ndenumerate(jb):
        pixar[5+2*index[0]]=value
        
      PixNested=int(''.join([str(score) for score in pixar]),2)
      PixRing=hp.nest2ring(1024,PixNested)
      mp[k,i,j]=m[PixRing]

In [ ]:
plt.imshow(mp[0])